In [2]:
!pip install praw


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [4]:
!pip install certifi


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import ssl
import certifi

# Force Python to use certifi's CA bundle
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())


In [2]:
import praw

# Create a Reddit instance (Replace with your credentials)
reddit = praw.Reddit(
    client_id="5XxI8AdauHxoDN8kgzck3w",
    client_secret="Y0PscnV3y40Kpz2kusRgnUPpB42Nww",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Function to fetch top posts and their top comments
def fetch_top_posts_with_comments(search_query, subreddit_name='all', post_limit=5, comment_limit=15):
    try:
        subreddit = reddit.subreddit(subreddit_name)
        top_posts = subreddit.search(search_query, sort='top', limit=post_limit)

        for post in top_posts:
            print(f"Post: {post.title}\n")
            
            # Fetch top comments
            post.comments.replace_more(limit=0)  # Load all comments
            top_comments = [comment.body for comment in post.comments.list()[:comment_limit]]
            
            for i, comment in enumerate(top_comments, 1):
                print(f"    Comment {i}: {comment}\n")
            print("\n------------------------\n")

    except Exception as e:
        print(f"Error fetching posts and comments: {e}")

# Main function
def main():
    search_query = input("Enter the topic you want to search for: ")
    subreddit_name = input("Enter the subreddit to search in (default is 'all'): ") or 'all'
    fetch_top_posts_with_comments(search_query, subreddit_name)

if __name__ == "__main__":
    main()


Post: It's a genuine question.

    Comment 1: At least for the PS5 Pro, WHY it exists is a relatively easy question to answer... Right?

WHY it exists specifically at THAT price though... Now that's the big question for Sony.

    Comment 2: >But why?

Because money.

    Comment 3: Why iphone 16 when you can wait another year and get iphone 17?

    Comment 4: It’s a filler episode year

    Comment 5: "We need your money, so I give you the same shit with slightly different but give me your money"

![gif](giphy|US86v9dnLweSaEQbq1|downsized)

    Comment 6: The iPhone 16 is just the yearly iPhone model, it exists as an incremental improvement over last year.

The PS5 Pro exists because money.

    Comment 7: ![gif](giphy|s239QJIh56sRW|downsized)

    Comment 8: Some diehard wanna-be rich technophobe kid :

![gif](giphy|am0dKpQO1vrOC3DmCb|downsized)

    Comment 9: Who’s the guy in the meme?

    Comment 10: People used to create great things and the selling point was that they'd last 

In [19]:
import praw
import json

# Create a Reddit instance (Replace with your credentials)
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Function to fetch top posts and their top comments
def fetch_top_posts_with_comments(search_query, subreddit_name='all', post_limit=5, comment_limit=15):
    data = []  # List to store post data
    try:
        subreddit = reddit.subreddit(subreddit_name)
        top_posts = subreddit.search(search_query, sort='top', limit=post_limit)

        for post in top_posts:
            post_data = {
                "title": post.title,
                "url": post.url,
                "comments": []
            }
            
            # Fetch top comments
            post.comments.replace_more(limit=0)  # Load all comments
            top_comments = [comment.body for comment in post.comments.list()[:comment_limit]]
            
            post_data["comments"].extend(top_comments)
            data.append(post_data)
        
        # Save data to a JSON file
        with open("reddit_posts.json", "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, indent=4, ensure_ascii=False)
        
        print("Data saved to reddit_posts.json")

    except Exception as e:
        print(f"Error fetching posts and comments: {e}")

# Main function
def main():
    search_query = input("Enter the topic you want to search for: ")
    subreddit_name = input("Enter the subreddit to search in (default is 'all'): ") or 'all'
    fetch_top_posts_with_comments(search_query, subreddit_name)

if __name__ == "__main__":
    main()

Data saved to reddit_posts.json


In [23]:
import praw
import requests
import spacy
import re
import ssl
from bs4 import BeautifulSoup

# Try to load Spacy model, use a fallback method if it fails
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic regex-based optimization.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)

        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts from the best subreddit
def fetch_reddit_posts(query):
    """Fetches top posts from the best subreddit for the query."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=5)

        results = []
        for post in top_posts:
            results.append({
                "title": post.title,
                "url": post.url,
                "comments": [comment.body for comment in post.comments[:5]]
            })
        
        return results
    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)

    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments']:
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


[WARNING] Spacy model not found. Using basic regex-based optimization.

🔍 Searching in subreddit: r/BudgetGamingLaptop for 'budget gaming laptop'


🔹 1. HOW IS THIS DEAL FOR A BUDGET GAMING LAPTOP?
🔗 URL: https://www.reddit.com/gallery/1ic181d
💬 Top Comments:
   - In Ca$ or Us$? In cad maybe but in usd NO!! I saw a RTX 4070m laptop for just 999 and with a far newer cpu
   - https://gaminglaptop.deals

🔹 2. Best Gaming Laptops in 2024/2025 List.
🔗 URL: https://www.reddit.com/r/BudgetGamingLaptop/comments/1emetuz/best_gaming_laptops_in_20242025_list/
💬 Top Comments:
   - [removed]
   - [removed]
   - Thanks for putting so much effort making this post. Definitely worth looking at it.

BTW, I use Predator and it's the best gaming laptop according to me, absolute beast.
   - I just picked up the Lenovo Legion 5 Pro with the AMD processor, and it’s a beast. It ticks all your boxes, but if you plan to game on it, you’ll likely need the power adapter. This is true for most gaming laptops.
   -

In [10]:
import praw

# Initialize Reddit API client
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Test if authentication works
print("Authentication successful:", reddit.read_only)  # Should print True


Authentication successful: True


In [24]:
import praw
import requests
import spacy
import re
import ssl
import json
from bs4 import BeautifulSoup

# Try to load Spacy model, use a fallback method if it fails
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic regex-based optimization.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)

        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts from the best subreddit
def fetch_reddit_posts(query):
    """Fetches top 10 posts from the best subreddit for the query and stores in a JSON file."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=10)

        results = []
        for post in top_posts:
            post.comments.replace_more(limit=0)  # Load all top-level comments
            comments = [comment.body for comment in post.comments[:20]]  # Get first 20 comments

            results.append({
                "title": post.title,
                "url": post.url,
                "comments": comments
            })
        
        # Save results to JSON file
        with open("reddit_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        print("✅ Data saved to reddit_results.json")

        return results

    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)

    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments'][:5]:  # Show only first 5 comments in the terminal
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


[WARNING] Spacy model not found. Using basic regex-based optimization.

🔍 Searching in subreddit: r/BudgetAudiophile for 'best gaming budget'

✅ Data saved to reddit_results.json

🔹 1. Best budget speakers against the wall
🔗 URL: https://www.reddit.com/r/BudgetAudiophile/comments/1ih26b3/best_budget_speakers_against_the_wall/
💬 Top Comments:
   - If you want them directly against the wall or maybe a few centimeters away, the only requirement is that they're not rear ported, as that would require merely a few inches, but not less.  Beyond that port spacing requirement, all speakers work best as as close to the wall as possible, but simply need EQ to tailor the bass response.  So buy whatever speakers you want based on the other factors you value or strengths they have, but you must implement a way of using parametric EQ, either from a PC, or miniDSP, or a source that has custom EQ ability or competent room correction.
   - You should look for front or downward ported speakers.   But eve